In [8]:
            import time
            import os
            import torch
            import torch.optim as optim
            from torch.utils.data import DataLoader
            from unet_anc_model import UNetANC  # Ensure this matches the path to your model definition file
            from dataset.dataset import NoisyCleanAudioDataset  # Ensure this matches the dataset class you're using
            import numpy as np
            import librosa

            # Update these paths with your dataset folders
            noisy_folder = "D:/ANC/dataset/pnoisy_trainset_28spk_wav"  # Replace with your noisy dataset path
            clean_folder = "D:/ANC/dataset/clean_trainset_28spk_wav"  # Replace with your clean dataset path

            # Ensure these paths are correct
            if not os.path.exists(noisy_folder) or not os.path.exists(clean_folder):
                raise FileNotFoundError(f"Dataset folders not found at {noisy_folder} or {clean_folder}. Ensure the paths are correct.")

            # **Model Training**
            def train_model():
                device = torch.device("cpu")  # For Intel Iris Xe GPU
                model = UNetANC().to(device)
                
                # Dataset loading
                dataset = NoisyCleanAudioDataset(noisy_folder, clean_folder)
                dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)
                
                # Optimizer and loss function
                optimizer = optim.Adam(model.parameters(), lr=0.001)
                criterion = torch.nn.MSELoss()
                
                num_epochs = 10  # Adjust the number of epochs as necessary
                
                # Start training
                print(f"Training on {device}...")
                start_time = time.time()
                
                for epoch in range(num_epochs):
                    epoch_start_time = time.time()  # Track time for each epoch
                    model.train()  # Set the model to training mode
                    
                    total_loss = 0
                    for i, (noisy, clean) in enumerate(dataloader):
                        noisy = noisy.to(device)
                        clean = clean.to(device)
                        
                        # Zero the gradients
                        optimizer.zero_grad()
                        
                        # Forward pass
                        output = model(noisy)
                        
                        # Calculate loss
                        loss = criterion(output, clean)
                        
                        # Backpropagation
                        loss.backward()
                        
                        # Update the weights
                        optimizer.step()
                        
                        # Track the loss for this batch
                        total_loss += loss.item()
                        
                        # Print loss at specified intervals
                        if (i + 1) % 100 == 0:  # Print every 100 batches
                            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(dataloader)}], Loss: {loss.item():.4f}")
                    
                    # Average loss for the epoch
                    avg_loss = total_loss / len(dataloader)
                    print(f"Epoch [{epoch + 1}/{num_epochs}] completed in {time.time() - epoch_start_time:.2f} seconds. Avg Loss: {avg_loss:.4f}")
                
                # Save the model after training
                model_save_path = "best_model_cpu.pth"
                torch.save(model.state_dict(), model_save_path)
                print(f"Training completed. Model saved as {model_save_path}. Total time: {time.time() - start_time:.2f} seconds.")

            if __name__ == "__main__":
                train_model()


Training on cpu...
Epoch [1/10], Step [100/724], Loss: 0.0091
Epoch [1/10], Step [200/724], Loss: 0.0061
Epoch [1/10], Step [300/724], Loss: 0.0039
Epoch [1/10], Step [400/724], Loss: 0.0029
Epoch [1/10], Step [500/724], Loss: 0.0026
Epoch [1/10], Step [600/724], Loss: 0.0019
Epoch [1/10], Step [700/724], Loss: 0.0022
Epoch [1/10] completed in 1660.94 seconds. Avg Loss: 0.0180
Epoch [2/10], Step [100/724], Loss: 0.0022
Epoch [2/10], Step [200/724], Loss: 0.0017
Epoch [2/10], Step [300/724], Loss: 0.0016
Epoch [2/10], Step [400/724], Loss: 0.0018
Epoch [2/10], Step [500/724], Loss: 0.0019
Epoch [2/10], Step [600/724], Loss: 0.0013
Epoch [2/10], Step [700/724], Loss: 0.0012
Epoch [2/10] completed in 1546.90 seconds. Avg Loss: 0.0016
Epoch [3/10], Step [100/724], Loss: 0.0015
Epoch [3/10], Step [200/724], Loss: 0.0010
Epoch [3/10], Step [300/724], Loss: 0.0010
Epoch [3/10], Step [400/724], Loss: 0.0009
Epoch [3/10], Step [500/724], Loss: 0.0015
Epoch [3/10], Step [600/724], Loss: 0.0007
E